In [ ]:
!pip install vk_api

In [2]:
import vk_api
import networkx as nx
from time import sleep


In [3]:
def auth_handler():
  """При двухфакторной аутентификации вызывается эта функция"""
  key = input("Enter authentication code: ")
  remember_device = True
  return key, remember_device

def captcha_handler(captcha):
  """ При возникновении капчи вызывается эта функция и ей передается объект
      капчи. Через метод get_url можно получить ссылку на изображение.
      Через метод try_again можно попытаться отправить запрос с кодом капчи
  """
  key = input("Enter captcha code {0}: ".format(captcha.get_url())).strip()
  return captcha.try_again(key)


def stop_f(items):
  print(items)

def auth(login:str, psswd:str):
  vk_session = vk_api.VkApi(login, psswd, auth_handler=auth_handler, captcha_handler=captcha_handler)
  try:
    vk_session.auth()
  except vk_api.AuthError as error_msg:
    print(error_msg)
  
  return vk_session

In [4]:
vk_session = auth("", "")
tools = vk_api.VkTools(vk_session)

In [5]:
def get_groups_users(friend_list, tools):
  friends_out = {}
  for friend in friend_list:
    try:
      friends_out[friend] = tools.get_all("friends.get", 100, {"user_id": friend})
    except Exception:
      friends_out[friend] = []
  sleep(1)
  return friends_out

In [ ]:
my_user_id = ###choose id
friends_list = [my_user_id]
friends_out = get_groups_users(friends_list, tools)
print(friends_out)

In [7]:
new_friends_list = [my_user_id]
for id in friends_out[my_user_id]['items']:
  new_friends_list.append(id)

In [8]:
friend_of_friend_list = get_groups_users(new_friends_list, tools)

In [9]:
def add_edge(f_item, s_item, graph):
  graph.add_edge(f_item, s_item)
  graph.add_edge(s_item, f_item)

In [10]:
G = nx.Graph()
for id in list(friend_of_friend_list):
  G.add_node(id)

In [11]:
for id in list(friend_of_friend_list):
  try:
    for friend_id in friend_of_friend_list[id]['items']:
      add_edge(id, friend_id, G)
  except:
    pass

In [12]:
nx.write_gexf(G, "my_graph.gexf")